In [1]:
import pandas as pd
from dicttoxml import dicttoxml
import xmltodict, json
import xml.etree.ElementTree as ET
from xml.dom import minidom
from jikanpy import Jikan
import time

In [2]:
NineAnimeList = pd.read_csv('./Data/animeListData/export.txt', names=['url'])
metaData = (NineAnimeList['url'].str.split('/')).apply(pd.Series)

anime = pd.DataFrame()

# The anime list dataset
anime['id'] = metaData[4]
anime['name'] = metaData[5]
anime['url'] = NineAnimeList['url']

In [3]:
# Required fields for the XML file. All of methods go into the MAL class.
class MAL():
    def __init__(self, user_id, user_name):
        self.user_id = user_id
        self.user_name = user_name
        self.jikan = Jikan()
        self.MALdict = None
        self.MALxml = None

    def animeDetails(self, id):
        # We will use the Jikan API with the anime id(s) to acquire additional information on the shows.
        # The jikan request rate limit is 1 request/2s (30 requests/minute).
        # I'll use 2.1s delay between requests just to be safe.
        # For 400 anime it should take exactly 14m
        time.sleep(2.1)
        series = self.jikan.anime(id)
        animeMeta = {
            'series_animedb_id': series['mal_id'],
            'series_title': series['title'],
            'series_type': series['type'],
            'series_episodes': series['episodes'],
            'my_id': 0,
            'my_watched_episodes': series['episodes'],
            'my_start_date': '0000-00-00',
            'my_finish_date': '0000-00-00',
            'my_rated': None,
            'my_score': None,
            'my_storage': None,
            'my_storage_value': 0.00,
            'my_status': 'Completed',
            'my_comments': None,
            'my_times_watched': 0,
            'my_rewatch_value': None,
            'my_priority': 'LOW',
            'my_tags': None,
            'my_rewatching': 0,
            'my_rewatching_ep': 0,
            'my_discuss': 1,
            'my_sns': 'default',
            'update_on_import': 1}
        return animeMeta

    def createDictList(self, animeIDs):
        myinfo = {'user_id': self.user_id,
                  'user_name': self.user_name,
                  'user_export_type': '1',
                  'user_total_anime': len(animeIDs),
                  'user_total_watching': '0',
                  'user_total_completed': len(animeIDs),
                  'user_total_onhold': '0',
                  'user_total_dropped': '0',
                  'user_total_plantowatch': '0'}

        # Collecting all of the details json files into a single list
        anime = []
        for seriesID in animeIDs:
            anime.append(self.animeDetails(seriesID))

        # Constructing the main object
        MALdict = {
            'myanimelist': {
                'myinfo': myinfo,
                'anime': anime}}

        self.MALdict = MALdict
        return MALdict

    def dictToMALxml(self, animeDict):
        # Converts a dict into an MAL xml file
        def itemName(arg):
            return 'anime'

        # A small bug caused by the inherent differences between json and xml datatypes.
        # In json you can't reuse a key twice, while the same is not true for xml file.
        animeDict['myanimelist']['CustomRemoval'] = animeDict['myanimelist'].pop(
            'anime')

        xml = dicttoxml(animeDict['myanimelist'], root=True, custom_root='myanimelist', ids=False,
                        attr_type=False, item_func=itemName)

        # Removing a key due the json->xml bug
        xml = xml.replace(b'<CustomRemoval>', b'')
        xml = xml.replace(b'</CustomRemoval>', b'')

        MALxml = minidom.parseString(xml).toprettyxml(indent="  ")
        self.MALxml = MALxml
        return MALxml
    
    def createMALxml(self, animeIDs):
        self.createDictList(animeIDs)
        self.dictToMALxml(self.MALdict)
        return self.MALxml

In [5]:
# Usage

# You don't actually need to specify an id or a username. 
# As of now the sites fixes the issue for you. So you can leave this as is.
mal = MAL(user_id='1234', user_name='John Joe')
MALxml = mal.createMALxml(anime['id'])

# Writting the MALxml into a file
with open("../MAL.xml", "w") as f:
    f.write(MALxml)

# Print to view the final output (or check the file directly)
print(MALxml)

<?xml version="1.0" ?>
<myanimelist>
  <myinfo>
    <user_id>1234</user_id>
    <user_name>John Joe</user_name>
    <user_export_type>1</user_export_type>
    <user_total_anime>40</user_total_anime>
    <user_total_watching>0</user_total_watching>
    <user_total_completed>40</user_total_completed>
    <user_total_onhold>0</user_total_onhold>
    <user_total_dropped>0</user_total_dropped>
    <user_total_plantowatch>0</user_total_plantowatch>
  </myinfo>
  <anime>
    <series_animedb_id>28171</series_animedb_id>
    <series_title>Shokugeki no Souma</series_title>
    <series_type>TV</series_type>
    <series_episodes>24</series_episodes>
    <my_id>0</my_id>
    <my_watched_episodes>24</my_watched_episodes>
    <my_start_date>0000-00-00</my_start_date>
    <my_finish_date>0000-00-00</my_finish_date>
    <my_rated/>
    <my_score/>
    <my_storage/>
    <my_storage_value>0.0</my_storage_value>
    <my_status>Completed</my_status>
    <my_comments/>
    <my_times_watched>0</my_times_watc